In [1]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np

import sys
import os
# Add the project root directory (not src) to sys.path
sys.path.append(os.path.abspath("../src/data/"))


clean = '../data/CLEAN'


Cofounder : "Box office" refers to total ticket sale revenues, and doesn't include revenues from streaming/VoD/etc. (which could be any amount and are usually unknown). 

0. 1991-2020 pour FBI data, ... pour GVD data
1. stantardiser les box-office 
2. plot les films au cours des années qui ont un box office au-dessus de la médianne
3. plot le genre qui ont le plus de box office 
4. plot le genre qui est le plus représenté dans les violent data (piechart)
5. plot les categories qui le plus représenté dans les violent data (piechart)
6. extract the words the most said in the summaries of the violent data 
7. plot the occurences of the words in the violent data 
8. Trouver unemployment data
9. Visualiser throughout the years real data (GVD, FBI)

# Preprocessing 

In [ ]:
# Save the DataFrame to a CSV file for the autoregressive model
Violent_Movies_autoregressive = pd.readcsv('../../data/CLEAN/Violent_Movies_final.tsv', sep="\t")
Kaggle_movies_metadata = pd.readcsv('../../data/CLEAN/Kaggle_movies_metadata_final.tsv', sep="\t")


TODO The loaded data must have no nan (fill with 0), corrspond to the years of the FBI

TODO Ensure that all have the same ylim !

In [ ]:
# Sort by year and week to ease further analysis 
Violent_Movies_autoregressive.sort_values(by=['Year', 'Week'], inplace=True)
# We are interested at non-zero box office revenue for visualisation purposes
Non_zero_Violent_Movies =Violent_Movies_autoregressive[Violent_Movies_autoregressive["Box office revenue"] != 0]

In [ ]:
Years_violent_movies = Violent_Movies_autoregressive.Year.unique()
Years_w_box_office = Non_zero_Violent_Movies.Year.unique()
print("The range of years for the violent movies that have a box office value is from {} to {}.".format(Years_w_box_office[0], Years_w_box_office[-1]))
print("The range of years that we have violent movies is from {} to {}.".format(Years_violent_movies[0], Years_violent_movies[-1]))

We decided to handle this difference of range with two approaches: 

- Considering only the movies that we have the box office value (First tests)
- Inputing the median of all box offices over the full period to the nan value (Jade)
- Inputing the rolling median over the number of weeks considered by our ARLD model as maximum (TODO)

Note that we tried to fill the missing values with the Kaggle Dataset as we did for the dates but disparities between the two datasets could be observed and thus the decision was made to stick only to CMU data to avoid creating inconsistencies and errors. 

Here is the attempt of filling with Kaggle dataset : 

In [ ]:
#iter on all dataset whethever having the value of box-office or not 

movie_w_box_office = Violent_Movies_autoregressive.copy()

counter_same = 0
counter_multiple = 0
counter_stay_same = 0
counter_diff = 0 
counter_less_than_1 = 0
counter_save = 0
counter_nothing = 0
diff_revenue = pd.DataFrame(columns=["Title","CMU_revenue", "Kaggle_revenue", "Diff","Percentage"])


for index, row in Violent_Movies_autoregressive.iterrows() :
    # is it in the kaggle dataset ?
    kaggle_data = Kaggle_movies_metadata[Kaggle_movies_metadata["title"] == row["Movie name"]]
    
    if kaggle_data.empty :
        if Violent_Movies_autoregressive.loc[index, "Box office revenue"]!= 0:
            counter_stay_same += 1
        else: 
            counter_nothing += 1
            movie_w_box_office.drop([index], inplace=True)

    #if yes and only ones 
    elif kaggle_data.shape[0] == 1 :

        if (kaggle_data.iloc[0]["revenue"] != 0) and (Violent_Movies_autoregressive.loc[index, "Box office revenue"] != 0):
            if  kaggle_data.iloc[0]["revenue"] == Violent_Movies_autoregressive.loc[index, "Box office revenue"]: 
                counter_same += 1
                #print("Revenue matched :) ")
        

            else: 
                Percentage_diff = np.abs(kaggle_data.iloc[0]["revenue"]-Violent_Movies_autoregressive.loc[index, "Box office revenue"])/(kaggle_data.iloc[0]["revenue"]+Violent_Movies_autoregressive.loc[index, "Box office revenue"])
                new_row = pd.DataFrame({"Title": [row["Movie name"]], "CMU_revenue": [Violent_Movies_autoregressive.loc[index, "Box office revenue"]],"Kaggle_revenue":[kaggle_data.iloc[0]["revenue"]], "Diff": np.abs(kaggle_data.iloc[0]["revenue"]-Violent_Movies_autoregressive.loc[index, "Box office revenue"]), "Percentage": Percentage_diff})
                diff_revenue = pd.concat([diff_revenue, new_row], ignore_index=True)
                counter_diff += 1
                if Percentage_diff < 0.2:
                    counter_less_than_1 += 1
                    print("Catch up in extremis")
                    Violent_Movies_autoregressive.loc[index, "Box office revenue"] = kaggle_data.iloc[0]["revenue"]
                else: 
                    movie_w_box_office.drop([index], inplace=True)

        elif (kaggle_data.iloc[0]["revenue"] != 0) and (Violent_Movies_autoregressive.loc[index, "Box office revenue"] == 0):
            counter_save += 1
            print("Save by kaggle")
            Violent_Movies_autoregressive.loc[index, "Box office revenue"] = kaggle_data.iloc[0]["revenue"]
        elif(kaggle_data.iloc[0]["revenue"] == 0) and (Violent_Movies_autoregressive.loc[index, "Box office revenue"] == 0):
            counter_nothing += 1
            movie_w_box_office.drop([index], inplace=True)
        
        elif(kaggle_data.iloc[0]["revenue"] == 0) and (Violent_Movies_autoregressive.loc[index, "Box office revenue"] != 0):
             counter_stay_same += 1

            


    else: #if multiple movies have this name in the kaggle dataset, compare with the incomplete date of the CMU dataset
        counter_multiple += 1
        date = Violent_Movies_autoregressive.loc[index, "Release date"]
        #if not the same year and not the year in the CMU, drop
        if pd.isna(date) or len(date) < 4 :
            movie_w_box_office.drop([index], inplace=True).drop([index],inplace=True)

        #  else, compare
        else :
            kaggle_data = kaggle_data[kaggle_data['release_date'].str[:4] == date[0:4]]
        #found only 1 movie this year ? extract the date !
            if kaggle_data.shape[0] == 1 :
                if ( kaggle_data.iloc[0]["revenue"] == Violent_Movies_autoregressive.loc[index, "Box office revenue"]) and (kaggle_data.iloc[0]["revenue"] != 0): 
                    counter_same += 1
                elif ( kaggle_data.iloc[0]["revenue"] != Violent_Movies_autoregressive.loc[index, "Box office revenue"]): 
                    if (kaggle_data.iloc[0]["revenue"] != 0) and (Violent_Movies_autoregressive.loc[index, "Box office revenue"] != 0):
                        counter_diff += 1
                        Percentage_diff = np.abs(kaggle_data.iloc[0]["revenue"]-Violent_Movies_autoregressive.loc[index, "Box office revenue"])/(kaggle_data.iloc[0]["revenue"]+Violent_Movies_autoregressive.loc[index, "Box office revenue"])
                        new_row = pd.DataFrame({"Title": [row["Movie name"]], "CMU_revenue": [Violent_Movies_autoregressive.loc[index, "Box office revenue"]],"Kaggle_revenue":[kaggle_data.iloc[0]["revenue"]], "Diff": np.abs(kaggle_data.iloc[0]["revenue"]-Violent_Movies_autoregressive.loc[index, "Box office revenue"]), "Percentage": Percentage_diff})
                        diff_revenue = pd.concat([diff_revenue, new_row], ignore_index=True)
                        if Percentage_diff < 0.2:
                            counter_less_than_1 += 1
                            print("Catch up in extremis")
                            Violent_Movies_autoregressive.loc[index, "Box office revenue"] = kaggle_data.iloc[0]["revenue"]
                        else: 
                            movie_w_box_office.drop([index], inplace=True)
                    elif (kaggle_data.iloc[0]["revenue"] != 0) and (Violent_Movies_autoregressive.loc[index, "Box office revenue"] == 0):
                        counter_save += 1
                        print("Save by kaggle")
                        Violent_Movies_autoregressive.loc[index, "Box office revenue"] = kaggle_data.iloc[0]["revenue"]
                    
                    elif(kaggle_data.iloc[0]["revenue"] == 0) and (Violent_Movies_autoregressive.loc[index, "Box office revenue"] != 0):
                        counter_stay_same += 1
                else:
                    counter_nothing += 1
                    movie_w_box_office.drop([index], inplace=True)
            else: 
                counter_nothing += 1
                movie_w_box_office.drop([index], inplace=True)
                


In [ ]:
print("There are {} movies that have less than 1 percents difference".format(counter_less_than_1))
print("There are {} matching revenues".format(counter_same))   
print("There are {} mismatched revenues, there is a difference between kaggle and cmu".format(counter_diff))
print("There are {} unmatch revenues because multiple countries have same title".format(counter_multiple))
print("There are {} movies that stay the same ".format(counter_stay_same))
print("There are {} movies that we have the information".format(counter_same+counter_stay_same))
print("There are {} movies that we have the information taking into account 1 percents difference".format(counter_same+counter_stay_same+counter_less_than_1))
print("There are {} violent movies in total".format(len(Violent_Movies_autoregressive)))
print("We can save {} movies".format(counter_save))
print("There are {} movies that we can save".format(counter_save+counter_same+counter_stay_same+counter_less_than_1))
print("It correspond to {:%} percentage of violent movies".format((counter_save+counter_same+counter_stay_same+counter_less_than_1)/len(Violent_Movies_autoregressive)))
print("All the counters {}".format(counter_save+counter_same+counter_stay_same+counter_diff+counter_nothing))
print("If we don't consider Kaggle dataset and only stick to the CMU one we have {:%} movies retained from the original dataset".format(len(Non_zero_Violent_Movies)/ len(Violent_Movies_autoregressive)*100))

Here is the different stantardisation and filling tested: 

In [ ]:
def normalise_box_office(extracted_data,window_size=6):

    rolling_mean = extracted_data["Box office revenue"].rolling(window=window_size, min_periods=1).mean()
    rolling_std = extracted_data["Box office revenue"].rolling(window=window_size, min_periods=1).std().fillna(1) # avoid division by 0 and enable to have 0 as z-score since z-score not applicable in that case
    extracted_data.loc[:, 'z_score'] = (extracted_data.count_of_offenses - rolling_mean ) / rolling_std

    return extracted_data

In [ ]:
def box_office_extracted(data,year,start_week,stop_week,ratio=False, normalised=False, window_size =6): 
    extracted_data = data[(data.year==year) & (data.week >=start_week) & (data.week <=stop_week)]
    if ratio:
        extracted_data = extracted_data["Box office revenue"]/extracted_data["Box office revenue"].max() # ratio on the max value TODO is it really the right way ?
    elif normalised:
        extracted_data = normalise_box_office(extracted_data,window_size)
    else:
        extracted_data = extracted_data["Box office revenue"]

    return extracted_data

In [ ]:
def aggregated_box_office(data,year, start_week, stop_week, ratio=False, normalised=False, window_size =6):
    extracted_data = box_office_extracted(data,year, start_week, stop_week, ratio, normalised, window_size)
    Violent_movie_metric = extracted_data.groupby(['Year', 'Week'])["Metric"].sum()
    return Violent_movie_metric

In [ ]:
# Example of use  
#aggregated_box_office(Non_zero_Violent_Movies,2008,8,10).to_csv(clean+"/movie_violence_metric.csv")

# Plots 

In [ ]:
# Plot on all the years and movies to obtain a first glimpse of the data
plt.figure(figsize=(20,10))
sns.boxplot(data=Non_zero_Violent_Movies["Box office revenue"])
plt.xlabel('Box office revenue')
plt.ylabel('Frequency')
plt.title('Box office revenue distribution of violent movies from 1991 to 2012')
plt.yscale('log')

In [ ]:
# Plot the distribution of box office revenue for violent movies from 1991 to 2012

# With Violin plot
plt.figure(figsize=(10, 6))
sns.catplot(
    data=Non_zero_Violent_Movies, x="Box office revenue", y="Release date", kind="violin",
)
plt.xscale('log')
plt.title('Box office revenue distribution of violent movies from 1991 to 2012')
plt.show()

In [ ]:
# With Box plot
sns.catplot(data=Non_zero_Violent_Movies, y="Box office revenue", x="Release date", kind="box")
plt.yscale('log')
# Rotating X-axis labels
plt.xticks(rotation = 90)
plt.show()


In [ ]:
# With Line plot
sns.relplot(data=Non_zero_Violent_Movies, x="Release date", y="Box office revenue", kind="line")
plt.xticks(rotation = 90)
plt.yscale('log')
plt.show()